# 多層パーセプトロン (MLP)
ニューラルネットワークの基本的な構造  
入力層、1つ以上の隠れ層（hidden layer）、出力層から構成され、線形変換 + 非線形活性化関数 により、データの特徴を学習する  
### パーセプトロン
単純な線形分類器
##  多層パーセプトロンの構造
### 1.入力層
データの特徴量を受け取る。特徴量の数が入力ノード数になる。
### 2.隠れ層
線形変換 を行った後、非線形活性化関数を適用。
### 3.出力層
回帰問題なら線形出力、分類問題なら softmax や sigmoid を適用
## 特徴
非線形データの学習が可能  
画像、テキスト、時系列データにも応用可能  
活性化間数の選択が重要
過学習しやすい→ドロップアウトやL2正則化を使う

In [2]:
import torch
import torch.nn as nn

#MLP モデルの定義
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # 隠れ層1
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)  # 出力層

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x



In [4]:
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#データの前処理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset  = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)

#モデル、損失関数、最適化手法の定義
input_dim = 28 * 28  # MNIST は 28x28 ピクセル
hidden_dim = 128
output_dim = 10  # 0~9 の 10 クラス
model = MLP(input_dim, hidden_dim, output_dim)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# **学習ループ**
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, 28*28)  # 画像を1次元化

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    #テスト精度の計算
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 28*28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Test Accuracy: {accuracy:.4f}")


Epoch [1/5], Test Accuracy: 0.9609
Epoch [2/5], Test Accuracy: 0.9722
Epoch [3/5], Test Accuracy: 0.9735
Epoch [4/5], Test Accuracy: 0.9750
Epoch [5/5], Test Accuracy: 0.9766
